In [ ]:
import duckdb
import pandas as pd

%load_ext sql

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
con = duckdb.connect(database = "dev.duckdb", read_only = True)

#### Evaluation of stg_receipts

In [8]:
receipts_df = con.sql('from stg_receipts').fetchdf()

In [11]:
# Check for null PK (receiptId) values
len(receipts_df[receipts_df['receiptId'].isnull()])

0

In [26]:
# Check for duplicate PK (receiptId) values
receipts_df['receiptId'].nunique(dropna=False) == len(receipts_df)

True

In [12]:
# Check for null FK (userId) values
len(receipts_df[receipts_df['userId'].isnull()])

0

In [ ]:
# Evaluate length of 'userId' attribute
receipts_df['userId'].str.len().value_counts()

userId
24    1119
Name: count, dtype: int64

In [25]:
# Check the recency of transactions
print(receipts_df['createDate'].max())
print(receipts_df['createDate'].min())

2021-03-01 16:17:34.772000-07:00
2020-10-30 14:17:59-06:00


#### Evaluation of stg_brands

In [27]:
brands_df = con.sql('from stg_brands').fetchdf()

In [28]:
# Check for duplicate PK (brandId) values
brands_df['brandId'].nunique(dropna=False) == len(brands_df)

True

In [29]:
# Check for duplicate FK (barcode) values
brands_df['barcode'].nunique(dropna=False) == len(brands_df)

False

In [30]:
# Display duplicate values
brands_size = brands_df.groupby('barcode').size().reset_index(name='record_count')

print(brands_size[brands_size['record_count'] > 1].sort_values(by='record_count', ascending=False))

          barcode  record_count
46   511111004790             2
282  511111204923             2
400  511111305125             2
622  511111504139             2
628  511111504788             2
745  511111605058             2
852  511111704140             2


#### Evaluation of stg_users

In [32]:
users_df = con.sql('from stg_users').fetchdf()

In [33]:
# Check for duplicate PK (userId) values
users_df['userId'].nunique(dropna=False) == len(users_df)

False

In [34]:
# Display duplicate values
users_size = users_df.groupby('userId').size().reset_index(name='record_count')

print(users_size[users_size['record_count'] > 1].sort_values(by='record_count', ascending=False))

                       userId  record_count
0    54943462e4b07e684157a532            20
11   5fc961c3b8cfca11a077dd33            20
3    59c124bae4b0299e55b0f330            18
8    5fa41775898c7a11a6bcef3e            18
34   5ff5d15aeb7c7d12096d91a2            18
..                        ...           ...
51   5ff873ddb3348b11c9337733             2
42   5ff73b90eb7c7d31ca8a452b             2
40   5ff7268eeb7c7d12096da2a9             2
9    5fb0a078be5fc9775c1f3945             2
203  60229990b57b8a12187fe9e0             2

[70 rows x 2 columns]


In [36]:
# Check the range of user creation dates
print(users_df['createdDate'].max())
print(users_df['createdDate'].min())

2021-02-12 07:11:06.240000-07:00
2014-12-19 07:21:22.381000-07:00


#### Evaluation of int_receiptItems

In [37]:
receipt_items_df = con.sql('from int_receiptItems').fetchdf()

In [40]:
for col_name, dtype in receipt_items_df.dtypes.items():
    print(f"{col_name}: {dtype}")

receiptId: object
userId: object
barcode: object
description: object
finalPrice: object
itemPrice: object
needsFetchReview: boolean
partnerItemId: object
preventTargetGapPoints: boolean
quantityPurchased: Int64
userFlaggedBarcode: object
userFlaggedNewItem: boolean
userFlaggedPrice: object
userFlaggedQuantity: Int64
needsFetchReviewReason: object
pointsNotAwardedReason: object
pointsPayerId: object
rewardsGroup: object
rewardsProductPartnerId: object
userFlaggedDescription: object
originalMetaBriteBarcode: object
originalMetaBriteDescription: object
brandCode: object
competitorRewardsGroup: object
discountedItemPrice: object
originalReceiptItemText: object
itemNumber: object
originalMetaBriteQuantityPurchased: Int64
pointsEarned: object
targetPrice: object
competitiveProduct: boolean
originalFinalPrice: object
originalMetaBriteItemPrice: object
deleted: boolean
priceAfterCoupon: object
metabriteCampaignId: object


In [41]:
# Identify if there are any barcode values missing in stg_brands
receipt_items_barcodes = set(receipt_items_df['barcode'].dropna())
brand_barcodes = set(brands_df['barcode'].dropna())
barcodes_not_in_brands = receipt_items_barcodes - brand_barcodes
print(len(barcodes_not_in_brands))

552
